In [4]:
import os
import numpy as np
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

csv=CSVLogger("1_rmsprop.log")
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1), activation = 'relu',strides=2,name='convo1'))
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))
# Step 3 - Flattening
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo4'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())
classifier.add(BatchNormalization(scale = False))
classifier.add(Dropout((0.5)))
classifier.add(Dense(1024, activation = 'relu'))
classifier.add(BatchNormalization(scale = False))
classifier.add(Dropout((0.4)))
classifier.add(Dense(20, activation = 'softmax'))


In [5]:
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

curr_path = os.getcwd()
basefolder = os.path.dirname(curr_path)
train_folder = os.path.join(basefolder, "Dataset\\Train")
test_folder = os.path.join(basefolder, "Dataset\\dev")

train_set = train_datagen.flow_from_directory(train_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

Found 2585 images belonging to 20 classes.
Found 200 images belonging to 20 classes.


In [10]:
history = classifier.fit(train_set,steps_per_epoch=train_set.n/32,epochs=10,validation_data=test_set,validation_steps=test_set.n/32,callbacks=[csv],verbose=1)

Epoch 1/10
80/80 [==============================] - 42s 521ms/step - loss: 0.4144 - accuracy: 0.8580 - val_loss: 2.8094 - val_accuracy: 0.2400
Epoch 2/10
80/80 [==============================] - 42s 524ms/step - loss: 0.3308 - accuracy: 0.8925 - val_loss: 4.8647 - val_accuracy: 0.1950
Epoch 3/10
80/80 [==============================] - 42s 521ms/step - loss: 0.2890 - accuracy: 0.9041 - val_loss: 5.7905 - val_accuracy: 0.2500
Epoch 4/10
80/80 [==============================] - 42s 525ms/step - loss: 0.2328 - accuracy: 0.9273 - val_loss: 2.8721 - val_accuracy: 0.4100
Epoch 5/10
80/80 [==============================] - 42s 524ms/step - loss: 0.1941 - accuracy: 0.9408 - val_loss: 10.0632 - val_accuracy: 0.2150
Epoch 6/10
80/80 [==============================] - 42s 517ms/step - loss: 0.1608 - accuracy: 0.9447 - val_loss: 5.7334 - val_accuracy: 0.3000
Epoch 7/10
80/80 [==============================] - 43s 527ms/step - loss: 0.1461 - accuracy: 0.9470 - val_loss: 3.3126 - val_accuracy: 0.350

In [12]:
classifier.save('1_rmsprop_new.h5')